In [ ]:
import sys
!{sys.executable} -m pip install --upgrade --force-reinstall git+https://github.com/arendjan/tmx-pico-aio.git@sensors
!{sys.executable} -m pip install nest_asyncio

Create a board and connect your components:

In [ ]:
import asyncio
import sys
import time
from tmx_pico_aio import tmx_pico_aio
import nest_asyncio
import struct
nest_asyncio.apply() # for jupyter notebook asyncio stuff

loop = asyncio.get_event_loop()

board = tmx_pico_aio.TmxPicoAio(loop=loop)

Add your callback, like printing or adding it to a list

In [ ]:
async def the_callback(data):
    # print(data)
    ints = list( map(lambda i:i.to_bytes(1, 'big'), data))
    bytes_obj = b''.join(ints)
    vals = list(struct.unpack('<9f', bytes_obj))
    print(f"x: {vals[0]: >8.2f} y: {vals[1]: >8.2f} z: {vals[2]: >8.2f} rx: {vals[3]: >8.2f} ry: {vals[4]: >8.2f} rz: {vals[5]: >8.2f} gx: {vals[6]: >8.2f} gy: {vals[7]: >8.2f} gz: {vals[8]: >8.2f}")


Add the sensor to the system

In [ ]:
async def addSensor(board):
    await asyncio.sleep(1)
    i2c_port = 0
    scl = 5
    sda = 4
    await board.set_pin_mode_i2c(i2c_port, sda, scl)
    await asyncio.sleep(0.1)
    await board.sensors.add_mpu9250(i2c_port, the_callback)

loop.run_until_complete(addSensor(board))



Run for some time

In [ ]:
async def run_till_end(board):
    # when to stop?
    await asyncio.sleep(10)
    await board.shutdown() # Otherwise next start will give you errors and you need to restart the kernel
    await asyncio.sleep(1)
    print("done")

loop.run_until_complete(run_till_end(board))